In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, NonlinearConstraint
from tqdm.notebook import tqdm
import os

In [2]:
np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.4f}'.format

In [3]:
M_0 = 5
m_0 = 0
STARTING_CAPITAL = 15
C_0 = 1_020_000
PROFIT_MARGIN = 0.2
AOV = 300

In [4]:
def probability_deliver(m_i):
    return 0.8 * (1.25 - np.exp(-1.0 * m_i/5))

In [5]:
def growth_after_spends(m_i, M_i):
    return (38/40) + ((0.45 + probability_deliver(m_i)) ** 0.3 + (M_i/(M_0 + 1)) ** 0.05) / 40

In [6]:
order_freq_week = pd.read_csv(os.path.join('..', 'input', 'customer-growth-strategy', 'order_frequency_data.csv'), index_col=0)

order_dist = order_freq_week.groupby('order_frequency_per_week', as_index=False)[['user_id']].count()

In [7]:
total_users_in_sample, total_orders_in_sample = order_freq_week.user_id.count(), np.dot(order_dist.order_frequency_per_week, order_dist.user_id)
total_users_in_sample, total_orders_in_sample

In [8]:
order_dist['weights'] = order_dist['user_id'] / order_dist['user_id'].sum()
order_dist['order_freq_per_day'] = order_dist['order_frequency_per_week'] / 7

In [9]:
def estimated_orders(customer_base_size):
    return np.sum(order_dist['weights'] * customer_base_size * order_dist['order_freq_per_day'])

estimated_orders(C_0)

In [10]:
def customer_base_growth(C_i_minus_1, m_i, M_i):
    return C_i_minus_1 * growth_after_spends(m_i, M_i)
customer_base_growth(C_0, m_0, M_0), customer_base_growth(C_0, 0, M_0) - C_0

In [11]:
def profit_on_day_i(customer_base_size):
    return estimated_orders(customer_base_size) * PROFIT_MARGIN * AOV / 10 ** 6
profit_on_day_i(C_0)

In [12]:
def create_df(capital_available_day_i_start, C_i_minus_1, m_i_arr, p, M_i_arr, growth, C_i, total_orders, profit, total_expenditure, capital_available_day_i_end):
    data = {
      "capital_available_day_i_start": capital_available_day_i_start,
      "C_i_minus_1": C_i_minus_1,
      "m_i_arr": m_i_arr,
      "p": p,
      "M_i_arr": M_i_arr,
      "growth": growth,
      "C_i": C_i,
      "total_orders": total_orders,
      "total_expenditure": total_expenditure,
      "profit": profit,
      "capital_available_day_i_end": capital_available_day_i_end,
    }
    return pd.DataFrame(data)

In [13]:
def obj_fun(x, n):
    m, M = x[:n].copy(), x[n:].copy()

    capital_available_day_i_start = [STARTING_CAPITAL]
    C_i_minus_1 = [None]
    m_i_arr = [m_0]
    p = [probability_deliver(m_0)]
    M_i_arr = [M_0]
    growth = [None]
    C_i = [C_0]
    total_orders = [estimated_orders(C_0)]
    total_expenditure = [m_0 + M_0]
    profit = [profit_on_day_i(C_0)]
    capital_available_day_i_end = [STARTING_CAPITAL - m_0 - M_0 + profit_on_day_i(C_0)]
    
    for i, (m_i, M_i) in enumerate(zip(m, M)):
        capital_available_day_i_start.append(capital_available_day_i_end[-1])
        C_i_minus_1.append(C_i[-1])
        m_i_arr.append(m_i)
        p.append(probability_deliver(m_i))
        M_i_arr.append(M_i)
        growth.append(growth_after_spends(m_i, M_i))
        C_i.append(customer_base_growth(C_i_minus_1[-1], m_i, M_i))
        total_orders.append(estimated_orders(C_i[-1]))
        total_expenditure.append(m_i_arr[-1] + M_i_arr[-1])
        profit.append(profit_on_day_i(C_i[-1]))
        capital_available_day_i_end.append(capital_available_day_i_start[-1] - m_i_arr[-1] - M_i_arr[-1] + profit[-1])

    return -1.0 * C_i[-1] / C_0

In [14]:
def obj_fun2(x, n):
    m, M = x[:n].copy(), x[n:].copy()

    capital_available_day_i_start = [STARTING_CAPITAL]
    C_i_minus_1 = [None]
    m_i_arr = [m_0]
    p = [probability_deliver(m_0)]
    M_i_arr = [M_0]
    growth = [None]
    C_i = [C_0]
    total_orders = [estimated_orders(C_0)]
    total_expenditure = [m_0 + M_0]
    profit = [profit_on_day_i(C_0)]
    capital_available_day_i_end = [STARTING_CAPITAL - m_0 - M_0 + profit_on_day_i(C_0)]
    
    for i, (m_i, M_i) in enumerate(zip(m, M)):
        capital_available_day_i_start.append(capital_available_day_i_end[-1])
        C_i_minus_1.append(C_i[-1])
        m_i_arr.append(m_i)
        p.append(probability_deliver(m_i))
        M_i_arr.append(M_i)
        growth.append(growth_after_spends(m_i, M_i))
        C_i.append(customer_base_growth(C_i_minus_1[-1], m_i, M_i))
        total_orders.append(estimated_orders(C_i[-1]))
        total_expenditure.append(m_i_arr[-1] + M_i_arr[-1])
        profit.append(profit_on_day_i(C_i[-1]))
        capital_available_day_i_end.append(capital_available_day_i_start[-1] - m_i_arr[-1] - M_i_arr[-1] + profit[-1])

    return create_df(capital_available_day_i_start, C_i_minus_1, m_i_arr, p, M_i_arr, growth, C_i, total_orders, profit, total_expenditure, capital_available_day_i_end)

In [15]:
x0 = [
    8.52585406381,
    8.60314910689,
    8.59057818804,
    8.57845150699,
    8.56626990527,
    8.55403092771,
    8.54173321437,
    8.52937876981,
    8.51696567713,
    8.50449576339,
    8.49196438326,
    8.47937670152,
    8.46672772487,
    8.45402020762,
    8.44125151373,
    8.42842268554,
    8.41553276725,
    8.40046105686,
    8.40549992596,
    8.41866575152,
    8.43387311104,
    8.44249197340,
    8.45845709272,
    8.47179845535,
    8.48518414076,
    8.49861496313,
    8.51208909370,
    8.52560855609,
    8.53917280717,
    8.55278200720,
    
    7.10046091076,
    7.11638527920,
    7.00254025677,
    6.98681767645,
    6.97093389039,
    6.95497102696,
    6.93885969709,
    6.92271415393,
    6.90656387572,
    6.89024469607,
    6.87384307727,
    6.85735113645,
    6.84076985348,
    6.82409884943,
    6.80733716928,
    6.79048524162,
    6.77354388201,
    6.76470180193,
    6.76783134861,
    6.78812071809,
    6.80653671540,
    6.83171036310,
    6.84970790723,
    6.87050037364,
    6.89142069290,
    6.91246907134,
    6.93364836516,
    6.95495758519,
    6.97639831601,
    6.99797144590,
]

In [16]:
obj_fun(x0, 30)

In [17]:
bounds = [(0.01,15) for _ in x0]

trust_constr_options = {
    'xtol': 0.01
}

def spend_less_than_available(x):
    df = obj_fun2(x, 30)
    res = df['capital_available_day_i_start'] - df['total_expenditure']
    return res

cons = [
#     {'type':'ineq', 'fun': lambda x: 15 - x.sum()}, # only leveraging the 15M we started with and nothing else
    NonlinearConstraint(spend_less_than_available, lb = 0, ub = 20),
]

In [19]:
best_fun = np.inf

for i in tqdm(range(100)):
    try:
        x0 = np.random.uniform(0.1, 20, 60)

        opt_results = minimize(
            obj_fun,
            x0=x0,
            args=(30),
            method="trust-constr",
            bounds=bounds,
            constraints=cons,
            tol=0.001,
            options=trust_constr_options
        )
        if opt_results['fun'] < best_fun:
            print(f"Current Optimal Value improved from {best_fun} to {opt_results['fun']}")
            best_opt_results = opt_results
            best_fun = opt_results['fun']
    except ValueError as VE:
        pass

In [20]:
best_opt_results['fun'], best_opt_results['x'], best_opt_results['success']

In [21]:
obj_fun2(best_opt_results['x'],30)